In [ ]:
import os
import glob
import subprocess

# Set this to the directory all of the sub### directories live in
studydir = '/mnt/hgfs/D/Data/Pigeon/Sleep'

hp = 100 # high pass filter
# Set this to the directory where you'll dump all the fsf files
# May want to make it a separate directory, because you can delete them all o
#   once Feat runs
fsfdir="%s/Scripts/fsfs3"%(studydir)

os.makedirs('%s/firstLevel_GLM'%(fsfdir))

# Get all the paths!  Note, this won't do anything special to omit bad subjects
subdirs=glob.glob("%s/analysis3/sub*/*/results"%(studydir))

for curr_sub in subdirs:
    print('Current subject is: %s \n'%(curr_sub))
    #TR = float(subprocess.getoutput('fslinfo %s/preprocessed_func_data.nii.gz | grep pixdim4 | head -n1 | awk \'{print $2}\'' % (curr_sub)))
    dim4 = float(subprocess.getoutput('fslinfo %s/preprocessed_data_highpassed_chopped.nii.gz | grep dim4 | head -n1 | awk \'{print $2}\'' % (curr_sub)))
    dim3 = float(subprocess.getoutput('fslinfo %s/preprocessed_data_highpassed_chopped.nii.gz | grep dim3 | head -n1 | awk \'{print $2}\'' % (curr_sub)))
    dim2 = float(subprocess.getoutput('fslinfo %s/preprocessed_data_highpassed_chopped.nii.gz | grep dim2 | head -n1 | awk \'{print $2}\'' % (curr_sub)))
    dim1 = float(subprocess.getoutput('fslinfo %s/preprocessed_data_highpassed_chopped.nii.gz | grep dim1 | head -n1 | awk \'{print $2}\'' % (curr_sub)))
    totalVoxels = dim1 * dim2 * dim3 * dim4
    
    splitdir = curr_sub.split('/')
    subNum = splitdir[-3][3:]
    date = splitdir[-2]
            
    replacements = {'SUBNUM':subNum , 'DATE':date, 'NTPS':str(dim4), 'TOTALVOX':str(totalVoxels)}
    
    with open("%s/template.fsf"%(fsfdir)) as infile: 
        with open("%s/firstLevel_GLM/design_sub%s_%s.fsf"%(fsfdir,subNum,date), 'w') as outfile:
            for line in infile:
            # Note, since the video, I've changed "iteritems" to "items"
            # to make the following work on more versions of python
            #  (python 3 no longer has iteritems())  
                for src, target in replacements.items():
                    line = line.replace(src, target)
                outfile.write(line)
   
    

# run first level analysis

In [ ]:
import os
import glob
import time
# Set this to the directory all of the sub### directories live in
datapath = '/mnt/hgfs/D/Data/Pigeon/Sleep' 


fsfs_files = glob.glob('%s/Scripts/fsfs/firstLevel_GLM/*.fsf'%(datapath))

for curr_fsf in fsfs_files[2:]:
    print('Running %s \n'%(curr_fsf.split('/')[-1]))
    results = os.system("feat %s"%(curr_fsf))
     

# copy reg folder to bold.feat to run group analysis

In [ ]:
import os
import glob
import time
# Set this to the directory all of the sub### directories live in
datapath = '/mnt/hgfs/D/Data/Pigeon/Sleep/analysis/'

sub_files = glob.glob('%s/sub*/*/results'%(datapath))
#reg_sources = glob.glob('%s/sub*/results/%s/reg'%(datapath,SeqName))

for curr_sub in sub_files:
    
    red_dir = os.path.join(curr_sub,'reg')
   
    reg_dist = os.path.join(curr_sub,'preprocessed_data_highpassed_chopped.feat','reg')
    print('Copying the reg folder to the feat directory of subject: %s'%(curr_sub.split('/')[-3:-1]))
    
    os.makedirs(reg_dist)
    os.system('cp -R %s/* %s'%(red_dir,reg_dist))
    
   